# Scrapping Tweets

## Translate using Built-in twitter translator

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd

# Connect to existing Chrome session
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()

# Start from a specific row (set this variable as needed)
start_from_row = 0

# Read the Excel file
# df = pd.read_excel(f'tweets.xlsx')
# df = pd.read_excel(f'translated_tweets.xlsx')

# Initialize the 'tweet-translated' column with False
# df['tweet-translated'] = False

# Set up a WebDriverWait instance for handling waits
wait = WebDriverWait(driver, 10)

# Iterate through DataFrame rows
for index, row in df.iterrows():
    if index < start_from_row:
        continue  # Skip to the starting row
    try:
        # Check if 'tweet-translate-after' is a string and contains 'View translation'
        if isinstance(row['tweet-translate-after'], str) and 'View translation' in row['tweet-translate-after']:
            # Navigate to the tweet URL
            driver.get(row['tweet-time href'])

            # Check if tweet is viewable
            tweet_tombstone = driver.find_elements(By.CLASS_NAME, 'tweet-tombstone')
            if tweet_tombstone:
                print(f"Tweet at row {index} is not viewable. Moving to the next row.")
                continue  # Skip to the next row as the tweet is not viewable

            # Wait for the page to load sufficiently
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'tweet-body-main')))

            # Click the 'View translation' button
            translate_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'tweet-translate')))
            translate_button.click()

            try:
                # Wait for the translation to appear with a timeout
                translated_text_element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'tweet-translated-text')), message="Translation not found within timeout")
                translated_text = translated_text_element.text
                
                # Write the translated tweet to 'tweet-body-translated' row
                df.at[index, 'tweet-body-translated'] = translated_text
                
                df.at[index, 'tweet-translated'] = True  # Indicate that this tweet has been translated
            except TimeoutException:
                print(f"Translation not found for tweet at row {index}")
        else:
            # print the current tweet-body-text to tweet-body-translated
            df.at[index, 'tweet-body-translated'] = row['tweet-body-text']
            df.at[index, 'tweet-translated'] = False
            
    except Exception as e:
        print(f"Error at row {index}: {e}")
        # Save the DataFrame state on error
        df.to_excel(f'error_at_row_{index}.xlsx', index=False)
        # Continue to the next row on error

# Close the driver
driver.quit()

# Save the DataFrame to a new Excel file
df.to_excel('translated_tweets.xlsx', index=False)

# Run on cmd
# "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222

Error at row 160: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF77A4582B2+55298]
	(No symbol) [0x00007FF77A3C5E02]
	(No symbol) [0x00007FF77A2805AB]
	(No symbol) [0x00007FF77A2C175C]
	(No symbol) [0x00007FF77A2C18DC]
	(No symbol) [0x00007FF77A2FCBC7]
	(No symbol) [0x00007FF77A2E20EF]
	(No symbol) [0x00007FF77A2FAAA4]
	(No symbol) [0x00007FF77A2E1E83]
	(No symbol) [0x00007FF77A2B670A]
	(No symbol) [0x00007FF77A2B7964]
	GetHandleVerifier [0x00007FF77A7D0AAB+3694587]
	GetHandleVerifier [0x00007FF77A82728E+4048862]
	GetHandleVerifier [0x00007FF77A81F173+4015811]
	GetHandleVerifier [0x00007FF77A4F47D6+695590]
	(No symbol) [0x00007FF77A3D0CE8]
	(No symbol) [0x00007FF77A3CCF34]
	(No symbol) [0x00007FF77A3CD062]
	(No symbol) [0x00007FF77A3BD3A3]
	BaseThreadInitThunk [0x00007FFEF4D2257D+29]
	RtlUserThreadStart [0x00007FFEF578AA58+40]

Error at row 240: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF77A4582B2+55298]
	(No symbol) [0x00007FF77A3C5E02]
	(No symbol) [0x00007FF77A2805AB]
	(N

KeyboardInterrupt: 

## Translate using google translate

I chose google translate because it capture the nuance of the sentece. Even mixed language one.

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd

# Connect to existing Chrome session
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")
driver = webdriver.Chrome(options=chrome_options)

# Start from a specific row (set this variable as needed)
start_from_row = 0

# Read the DataFrame
df = pd.read_excel('tweets_google_translate.xlsx')  # Replace 'tweets.xlsx' with your actual file

# Set up a WebDriverWait instance for handling waits
wait = WebDriverWait(driver, 10)

# Go to Google Translate page
driver.get("https://www.google.com/search?q=translate+malay+to+english")

# Iterate through DataFrame rows
for index, row in df.iterrows():
    if index < start_from_row:
        continue  # Skip to the starting row
    try:
        # Input the text into the specified Google Translate textarea
        input_area = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'textarea#tw-source-text-ta')))
        input_area.clear()
        input_area.send_keys(row['tweet-body-text'])

        # Wait for the translation indicator to change class, signaling completion
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.RxYbNe.iRZc1e')))

        # Scrape the translated text
        translated_text_element = driver.find_element(By.CSS_SELECTOR, 'pre#tw-target-text span.Y2IQFc')
        translated_text = translated_text_element.text

        # Write the translated tweet to 'tweet-body-translated' row
        df.at[index, 'tweet-body-translated'] = translated_text

    except Exception as e:
        print(f"Error at row {index}: {e}")
        # Save the DataFrame state on error
        df.to_excel(f'error_at_row_{index}.xlsx', index=False)
        # Continue to the next row on error

# Close the driver
driver.quit()

# Save the DataFrame to a new Excel file
df.to_excel('tweets_google_translated.xlsx', index=False)

# Run on cmd
# "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222

C:\Users\Haziq\AppData\Local\Temp\ipykernel_29804\3951963909.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Translation' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'tweet-body-translated'] = translated_text


# Data Cleaning

## Fix mistranslation of "Mat Kilau"

In [10]:
import re

# Re-load the DataFrame to ensure we're working with the latest data
df = pd.read_excel('tweets_google_translated.xlsx')

# Define a function to correct mistranslations while considering all permutations
def correct_mistranslations(text):
    # Patterns to identify all permutations of the mistranslated words
    patterns = [
        r'mat\s?shines?', r'mat\s?glossy', r'matte\s?shines?', r'matte\s?glossy',
        r'shines?\s?mat', r'glossy\s?mat', r'shines?\s?matte', r'glossy\s?matte',
        r'matte\s?gloss', r'gloss\s?mat', r'glossy\s?mat', r'glossy',
        r'shines?', r'gloss', r'matte', r'shiny\s?mat', r'mat\s?kailau', r'shiny\s?mats',
        r'mats', r'shiny', r'mat\s?luster?', r'glitter\s?mat?'
    ]
    
    # Combine the patterns into a single regular expression
    combined_pattern = r'(' + '|'.join(patterns) + r')'
    
    # Function to replace matched patterns with 'Mat Kilau'
    def replace_with_mat_kilau(match):
        return 'Mat Kilau'
    
    # Replace all found patterns with 'Mat Kilau', case-insensitively
    corrected_text = re.sub(combined_pattern, replace_with_mat_kilau, text, flags=re.IGNORECASE)
    
    return corrected_text

# Apply the correction function to the 'tweet-body-translated' column
df['tweet-body-translated-corrected'] = df['tweet-body-translated'].apply(correct_mistranslations)

df.to_excel('tweets_google_translated_fixmat.xlsx', index=False)


In [46]:
# Load the new Excel file
cleaned_file_path = 'tweets_google_translated_fixmat.xlsx'
cleaned_tweets_df = pd.read_excel(cleaned_file_path)

# Checking for rows where 'tweet-body-translated-corrected' does not contain 'Mat Kilau'
rows_without_mat_kilau = cleaned_tweets_df[~cleaned_tweets_df['tweet-body-translated-corrected'].str.contains('Mat Kilau', case=False, na=False)]

# Displaying the rows that do not contain 'Mat Kilau'
rows_without_mat_kilau[['tweet-body-text', 'tweet-body-translated-corrected']]

,tweet-body-text,tweet-body-translated-corrected
9,sorry tak pernah tgk sampai habis movie mat ki...,sorry I never watched the movie until the end....
24,Mende babi tengok mat kilau nangis baik tengok...,"When you look at the pig, it’s good to cry whe..."
55,Hahaha aku mat kilau pun tak tengok . Apatah l...,Hahaha I can’t even see it. Not to mention the...
63,"Tngk mat kilau the movie je lah,mcmana org cid...","Let’s just look at the movie, how do people wh..."
70,X dak masalah.. masing2 boleh p.. x mau sudah....,It’s no problem.. you can do it.. you don’t wa...
...,...,...
1264,Haha lepas mat kilau keluar kat netflix org ba...,"Haha, after it came out on netflix, people jus..."
1290,"""tak best la cite ni bunyi dia over"" my bro to...","""It’s not the best la cite, it sounds like it’..."
1300,mama as cikgu sejarah said something wrong w m...,mama as a history teacher said something wrong...
1314,Klu asa tgk mat kilau mcm nk brsilat. Tp tgk a...,I hope you don’t lose the luster like you do i...


In [13]:
import pandas as pd
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

# Assuming the CSV file is named 'tweets.csv' and it has a column named 'tweet-body-text'
# Since the actual file is not available, this is a demonstration of how the code would look

try:
    tweets_df = pd.read_excel('tweets_google_translated_fixmat.xlsx')
except UnicodeDecodeError as e:
    print("Error reading file:", e)

In [14]:
nltk.download('stopwords')
# Get English stopwords from NLTK
stop_words = set(stopwords.words('english'))

# Assuming 'tweets_df' is your DataFrame after reading the CSV
tweets = tweets_df['tweet-body-translated-corrected'].tolist()
all_tweets = ' '.join(tweets)
clean_tweets = re.sub(r'[^\w\s]', '', all_tweets.lower())  # Remove punctuation
clean_tweets = re.sub(r'\d+', '', clean_tweets)  # Remove numbers
words = clean_tweets.split()

# Filter out stop words
filtered_words = [word for word in words if word not in stop_words]

# Count the frequency of each word
word_counts = Counter(filtered_words)

top_30 = word_counts.most_common(50)
# Get the top 10 most common words

top_30

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Haziq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[('mat', 1338),
 ('movie', 1280),
 ('kilau', 1221),
 ('like', 393),
 ('dont', 311),
 ('people', 305),
 ('movies', 285),
 ('watch', 215),
 ('want', 178),
 ('malay', 173),
 ('netflix', 150),
 ('best', 133),
 ('even', 131),
 ('watching', 129),
 ('see', 124),
 ('im', 121),
 ('story', 120),
 ('many', 119),
 ('good', 118),
 ('make', 112),
 ('know', 105),
 ('say', 102),
 ('time', 101),
 ('one', 101),
 ('film', 99),
 ('watched', 98),
 ('bad', 88),
 ('thats', 86),
 ('history', 85),
 ('think', 85),
 ('really', 82),
 ('still', 76),
 ('also', 74),
 ('lot', 73),
 ('didnt', 64),
 ('look', 59),
 ('doesnt', 57),
 ('much', 54),
 ('better', 54),
 ('get', 54),
 ('historical', 53),
 ('first', 51),
 ('right', 50),
 ('malays', 49),
 ('said', 49),
 ('fight', 49),
 ('cant', 48),
 ('camera', 48),
 ('cinema', 46),
 ('scene', 46)]